In [ ]:
import numpy as np
import os
import torch
import torch.nn as nn
import time
import pandas as pd
import json
from scipy.stats import pearsonr

In [ ]:
from model.util import Normalizer, load_column_stats
from model.database_util import get_hist_file, get_job_table_sample, collator
from model.model import QueryFormer
from model.database_util import Encoding
from model.dataset import PlanTreeDataset
from model.trainer import eval_workload, train

In [ ]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 128
    lr = 0.001
    # epochs = 200
    epochs = 100
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    device = 'cuda:0'
    newpath = './results'
    to_predict = 'card'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [ ]:
encoding = Encoding()

In [ ]:
# 加载列统计信息
column_stats_path = f'./data/column_min_max_vals.csv'
column_min_max_vals, col2idx = load_column_stats(column_stats_path)
    
encoding = Encoding(
        column_min_max_vals=column_min_max_vals,
        col2idx=col2idx,
        op2idx={'>':0, '=':1, '<':2, 'NA':3}
    )

In [ ]:
# hist_file = get_hist_file(data_path + 'histogram_string.csv')
cost_norm = Normalizer(-3.61192, 12.290855)
card_norm = Normalizer(1,100)

/home/shaikhq/QueryFormer/model/database_util.py:76: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  hist_file['freq'][i] = freq_np
/home/shaikhq/QueryFormer/model/database_util.py:89: FutureWarning: ChainedAssignmentError: behaviour will chan

In [7]:
from model.util import seed_everything
seed_everything()

In [ ]:
model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = False, use_hist = False, \
                 pred_hid = args.pred_hid
                )

In [9]:
_ = model.to(args.device)

In [ ]:
to_predict = 'card'

In [ ]:
with open('train_data.json', 'r', encoding='utf-8') as f:
    train_list = json.load(f)

with open('valid_data.json', 'r', encoding='utf-8') as f:
    val_list = json.load(f)

In [ ]:
train_ds = PlanTreeDataset(train_list, None, encoding, None, card_norm, cost_norm, to_predict, None)
val_ds = PlanTreeDataset(val_list, None, encoding, None, card_norm, cost_norm, to_predict, None)

In [ ]:
crit = nn.MSELoss()
model, best_path = train(model, train_ds, val_ds, crit, card_norm, args)

OutOfMemoryError: CUDA out of memory. Tried to allocate 46.00 MiB. GPU 0 has a total capacity of 3.94 GiB of which 3.06 MiB is free. Including non-PyTorch memory, this process has 3.91 GiB memory in use. Of the allocated memory 3.70 GiB is allocated by PyTorch, and 146.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Do evaluation
methods = {
    'encoding': encoding,
    'cost_norm': card_norm,
    'hist_file': None,
    'model': model,
    'device': args.device,
    'bs': 512,
}

In [ ]:
_ = eval_workload('job-light', methods)

In [ ]:
_ = eval_workload('synthetic', methods)